<a href="https://colab.research.google.com/github/havihaviplants/Machine-learning/blob/main/cnn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [ ]:
# 자신의 Google Drive 마운트하는 코드를 추가하자!
mnist = np.load('/content/drive/MyDrive/MY Drive/mnist.npz')
x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
# MNIST 이미지의 크기를 4배로 확장
# ImageNet에서 활용했던 네트워크 사용을 위해...
x_train_large = np.repeat(x_train, 2, axis=1)
x_train_large = np.repeat(x_train_large, 2, axis=2)
x_test_large = np.repeat(x_test, 2, axis=1)
x_test_large = np.repeat(x_test_large, 2, axis=2)
print(x_train_large.shape, x_test_large.shape)


(60000, 56, 56) (10000, 56, 56)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train_large), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test_large), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(

            nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
class BasicBlock(nn.Module):
 expansion = 1
 def __init__(self, in_channels, out_channels, stride=1):
  super().__init__()
  self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
  self.bn1 = nn.BatchNorm2d(out_channels)
  self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
  self.bn2 = nn.BatchNorm2d(out_channels)
  self.shortcut = nn.Sequential()
  if stride != 1 or in_channels != self.expansion * out_channels:
   self.shortcut = nn.Sequential(
    nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
    nn.BatchNorm2d(self.expansion * out_channels)

   )
 def forward(self, x):
  out = F.relu(self.bn1(self.conv1(x)))
  out = self.bn2(self.conv2(out))
  out += self.shortcut(x)
  out = F.relu(out)
  return out


In [ ]:
class ResNet(nn.Module):
 def __init__(self, block, num_blocks, num_classes=10): # 원래는 1000이나, MNIST 데이터셋을 활용하므로 10으로 변경
  super().__init__()
  self.in_channels = 64
  # 원래는 3인데, MNIST 데이터셋을 활용하므로 1로 변경
  self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
  self.bn1 = nn.BatchNorm2d(64)
  self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
  self.layer2 = self.make_layer(block, 128, num_blocks[1], stride=2)
  self.layer3 = self.make_layer(block, 256, num_blocks[2], stride=2)
  self.layer4 = self.make_layer(block, 512, num_blocks[3], stride=2)
  self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
  self.fc = nn.Linear(512 * block.expansion, num_classes)

 def make_layer(self, block, out_channels, num_blocks, stride):
  strides = [stride] + [1] * (num_blocks - 1)
  layers = []
  for stride in strides:
   layers.append(block(self.in_channels, out_channels, stride))
   self.in_channels = out_channels * block.expansion
  return nn.Sequential(*layers)

 def forward(self, x):
  out = F.relu(self.bn1(self.conv1(x)))
  out = self.layer1(out)
  out = self.layer2(out)
  out = self.layer3(out)
  out = self.layer4(out)
  out = self.avg_pool(out)
  out = out.view(out.size(0), -1)
  out = self.fc(out)
  return out
def ResNet18(num_classes=10):
 return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

In [ ]:
from torch.optim import Adam
# 사용하고자 하는 모델의 주석을 풀어서 활용하자!
# model = AlexNet().to(device)
#model = VGG16().to(device)
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss().to(device)
opti = Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm
def train(model, dataloader, criterion, data_len, opti):
 correct = 0

 model.train()
 for data, target in tqdm(dataloader):
  data = data.view(-1, 1, 56, 56).to(device)
  target = target.to(device)

  output = model(data)
  loss = criterion(output, target)

  opti.zero_grad()
  loss.backward()
  opti.step()

  pred = output.max(1, keepdim=True)[1]
  correct += pred.eq(target.view_as(pred)).sum().item()

 acc = 100. * correct / data_len
 return acc

In [ ]:
def evaluate(model, dataloader, criterion, data_len):
 correct = 0

 model.eval()
 for data, target in dataloader:
  data = data.view(-1, 1, 56, 56).to(device)
  target = target.to(device)

  output = model(data)
  loss = criterion(output, target)

  pred = output.max(1, keepdim=True)[1]
  correct += pred.eq(target.view_as(pred)).sum().item()

 acc = 100. * correct / data_len
 return acc


In [ ]:
epoch = 2
for i in range(epoch):
 train_acc = train(model, train_dataloader, criterion, len(train_dataloader.dataset), opti)
 val_acc = evaluate(model, test_dataloader, criterion, len(test_dataloader.dataset))
 print(f"[Epoch: {i:2d}], [Train Acc: {train_acc:3.4f}], [Val Acc: {val_acc:3.4f}]")

100%|██████████| 15000/15000 [04:46<00:00, 52.42it/s]


[Epoch:  0], [Train Acc: 96.3800], [Val Acc: 99.2900]


100%|██████████| 15000/15000 [04:42<00:00, 53.16it/s]


[Epoch:  1], [Train Acc: 98.5767], [Val Acc: 99.0600]
